In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import torch.nn as nn


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

CNN_MODEL_PATH = "best_model_efficientnet.pth"
# Load model dengan pretrained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def load_cnn_model():
    model = models.efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.load_state_dict(torch.load(CNN_MODEL_PATH, map_location=device))
    return model.to(device).eval()

cnn_model = load_cnn_model()
print("Model berhasil dimuat!")

d:\ComputerVision\belajarConda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\ComputerVision\belajarConda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model berhasil dimuat!


In [3]:
# Transformasi gambar agar sesuai dengan model (misalnya, ukuran 224x224 untuk MobileNetV2)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisasi sesuai ImageNet
])

def load_image(image_path):
    """Load dan preproses gambar"""
    image = Image.open(image_path).convert("RGB")  # Buka gambar
    image = transform(image)  # Terapkan transformasi
    image = image.unsqueeze(0)  # Tambahkan batch dimension
    return image.to(device)


In [11]:
def predict(image_path):
    """Fungsi untuk melakukan prediksi pada gambar baru"""
    image = load_image(image_path)
    
    with torch.no_grad():
        output = cnn_model(image)
        _, predicted = torch.max(output, 1)
    
    class_names = ["Memakai-Helm", "Tidak-Memakai-Helm"]  # Sesuai urutan pelatihan
    print(f"Prediksi: {class_names[predicted.item()]}")
    
    return class_names[predicted.item()]

# Contoh penggunaan
image_path = "D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\pelanggar\\violation_20250323_203332168590.jpg"  # Ganti dengan path gambar yang ingin diuji
prediction = predict(image_path)

Prediksi: Tidak-Memakai-Helm
